In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [30]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

def get_image(soup):
    try:
        image = soup.find("img", attrs={"id": "landingImage"})["src"]
    except AttributeError:
        image = ""
    return image

def get_link(soup):
    try:
        link = soup.find("a", attrs={'class': 'a-link-normal s-no-outline'})['href']
        full_link = "https://www.amazon.com" + link
    except AttributeError:
        full_link = ""
    return full_link

In [31]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=fertilizer&crid=XOX4HO870Z7Q&sprefix=fertilize%2Caps%2C372&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))

    d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": [], "image": [], "link": []}

    # Loop for extracting product details from each link
    for link in links_list:
        full_link = "https://www.amazon.com" + link
        new_webpage = requests.get(full_link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['image'].append(get_image(new_soup))
        d['link'].append(full_link)

    # Create a DataFrame
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])

    # Save to CSV
    amazon_df.to_csv("amazon_fert.csv", header=True, index=False)

C:\Users\abhir\AppData\Local\Temp\ipykernel_24924\2249737546.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [32]:
amazon_df

,title,price,rating,reviews,availability,image,link
0,100 EarthPods - Premium Flower Fertilizer Spik...,,4.5 out of 5 stars,"1,853 ratings",In Stock,https://m.media-amazon.com/images/I/81vh-on2+N...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1,"Miracle-Gro Indoor Plant Food, Liquid Plant Fo...",,4.6 out of 5 stars,"33,368 ratings",Not Available,https://m.media-amazon.com/images/I/71cTl+fiDm...,https://www.amazon.com/Miracle-Gro-VB300526-Li...
2,Burpee Natural Purpose Granular 4-Lb Organic F...,,4.6 out of 5 stars,"18,305 ratings",Not Available,https://m.media-amazon.com/images/I/71nL-dLYxD...,https://www.amazon.com/Burpee-Natural-Organic-...
3,SUPERthrive The Original Vitamin Solution - Li...,,4.7 out of 5 stars,"24,482 ratings",In Stock,https://m.media-amazon.com/images/I/71uKU70eMR...,https://www.amazon.com/SUPERthrive-VI30148-Pla...
4,Down to Earth All Natural Fertilizers Organic ...,,4.6 out of 5 stars,"3,353 ratings",Not Available,https://m.media-amazon.com/images/I/51vBZRiCxU...,https://www.amazon.com/Down-Earth-Organic-Citr...
...,...,...,...,...,...,...,...
60,Big Foot Mycorrhizae - Double Your Plant Size ...,,4.6 out of 5 stars,897 ratings,Not Available,https://m.media-amazon.com/images/I/51rvR1duUG...,https://www.amazon.com/Big-Foot-Mycorrhizae-Gr...
61,32275 4LB Triple Super Phosphate,,4.7 out of 5 stars,202 ratings,In Stock,https://m.media-amazon.com/images/I/61QrtfmhVu...,https://www.amazon.com/VOLUNTARY-PURCHASING-32...
62,"4X Mycorrhizae Rooting Hormone, Mycorrhizal Fu...",,4.4 out of 5 stars,100 ratings,In Stock,https://m.media-amazon.com/images/I/51ETMQYtv-...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
63,Pennington Rejuvenate Citrus & Avocado 4 Pounds,,4.4 out of 5 stars,34 ratings,In Stock,https://m.media-amazon.com/images/I/71weiMz29z...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
